In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [23]:
dataset = pd.read_csv('data/flight_info.csv')

In [24]:
data = pd.read_csv('data/T_ONTIME_REPORTING2.csv')
data['FL_DATE'] = pd.to_datetime(data['FL_DATE'])
data['DAY_OF_WEEK'] = data['FL_DATE'].dt.day_name()
data = data.drop(columns=['FL_DATE'])

/var/folders/jq/51d8jm5x4v5crly84p0n52qr0000gn/T/ipykernel_89792/1312736740.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['FL_DATE'] = pd.to_datetime(data['FL_DATE'])


In [25]:
data_trim = data [['OP_CARRIER_AIRLINE_ID', 'ORIGIN', 'DEST', 'DAY_OF_WEEK', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'AIR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE','DEP_DELAY', 'ARR_DELAY', 'TAXI_OUT', 'TAXI_IN', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']]

In [26]:
data_trim = data_trim.fillna(0)

In [27]:
data_trim['TOTAL_DELAY'] = data_trim[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].sum(axis=1)


In [28]:
data_trim['IS_DELAY'] = np.where(data_trim['TOTAL_DELAY'] > 0, 1, 0)



In [29]:
dataset = data_trim[['OP_CARRIER_AIRLINE_ID', 'ORIGIN', 'DEST', 'DAY_OF_WEEK', 'CRS_DEP_TIME', 'CRS_ARR_TIME','AIR_TIME', 'CRS_ELAPSED_TIME','DISTANCE','DEP_DELAY', 'ARR_DELAY', 'TAXI_OUT', 'TAXI_IN', 'IS_DELAY', 'TOTAL_DELAY']]


In [30]:
dataset.to_csv('data/flight_info.csv')

In [16]:
# Splitting the data into training and testing sets
X = dataset[['AIR_TIME', 'DISTANCE','CRS_ELAPSED_TIME']]
y = dataset[['TOTAL_DELAY', 'IS_DELAY']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=75)

# Scaling the data
scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [17]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)
score, accuracy = model.evaluate(X_test, y_test, verbose=0)

Epoch 1/5


/Users/neha/Documents/SCHOOL/Fall24/DSBA_6156/6156_project/6156/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15279/15279 ━━━━━━━━━━━━━━━━━━━━ 9s 528us/step - accuracy: 0.9207 - loss: 3724.7729
Epoch 2/5
15279/15279 ━━━━━━━━━━━━━━━━━━━━ 12s 772us/step - accuracy: 0.9993 - loss: 1800.9270
Epoch 3/5
15279/15279 ━━━━━━━━━━━━━━━━━━━━ 8s 538us/step - accuracy: 0.9982 - loss: 1823.6481
Epoch 4/5
15279/15279 ━━━━━━━━━━━━━━━━━━━━ 8s 510us/step - accuracy: 0.9978 - loss: 1767.9924
Epoch 5/5
15279/15279 ━━━━━━━━━━━━━━━━━━━━ 9s 560us/step - accuracy: 0.9977 - loss: 1740.3047


In [19]:
user_input = np.array([[309,2475,372]])
#user_scaled = scaler.transform(user_input)

In [20]:
model.predict(user_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


array([[16.306482 ,  0.2143361]], dtype=float32)

In [21]:
model.save('nn_model.keras')